In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jaclyn.bnv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jaclyn.bnv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jaclyn.bnv\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# Read in iris_raw.csv
df = pd.read_csv("../data/customer_prepped.csv")
# Build X, y (target -> y), drop species if present
y = df["churn"]
# X = df.drop(columns=["churn",'product_id','sale_id', 'customer_id', 'price', 'total_value','feedback_text'], errors="ignore")
X = df.drop(columns=["tokens","churn",'product_id','sale_id','feedback_text','sentiment', 'sale_date', 'last_purchase_date','customer_id'], errors="ignore")

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
log_model = LogisticRegression(max_iter=500, class_weight="balanced")
param_grid = {"C": [0.01, 0.1, 1, 10, 100]}
search = GridSearchCV(log_model, param_grid, cv=5, scoring="roc_auc")
search.fit(X_train, y_train)
print("Best C:", search.best_params_)
print("Best CV AUC:", search.best_score_)


c:\Users\jaclyn.bnv\OneDrive - Incedo Technology Solutions Ltd\Documents\incedo-capstone\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\jaclyn.bnv\OneDrive - Incedo Technology Solutions Ltd\Documents\incedo-capstone\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max

Best C: {'C': 100}
Best CV AUC: 0.5462762567491938


c:\Users\jaclyn.bnv\OneDrive - Incedo Technology Solutions Ltd\Documents\incedo-capstone\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
# Best model after tuning
best_model = search.best_estimator_

# Predictions
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:, 1]

In [25]:
# Metrics
acc = accuracy_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_prob)

print("Best C:", search.best_params_)
print("Best CV AUC:", search.best_score_)
print("Test Accuracy:", acc)
print("Test ROC AUC:", roc)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Best C: {'C': 100}
Best CV AUC: 0.5462762567491938
Test Accuracy: 0.51875
Test ROC AUC: 0.5147637881784223

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.52      0.63      1230
           1       0.24      0.51      0.33       370

    accuracy                           0.52      1600
   macro avg       0.51      0.51      0.48      1600
weighted avg       0.65      0.52      0.56      1600



In [26]:
with open("../models/log_reg.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("Model saved to models/log_reg.pkl")

Model saved to models/log_reg.pkl
